<a href="https://colab.research.google.com/github/quantam665/reddit-persona-generator1/blob/main/reddit_persona_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🚀 Reddit Persona Generator using Gemini 2.0 Flash (One-Cell Script)

# 📦 Install required packages
!pip install -q google-generativeai praw

# 🔐 Fill in your API credentials here
REDDIT_CLIENT_ID = 'your_reddit_client_id'
REDDIT_SECRET = 'your_reddit_secret'
REDDIT_USERNAME = 'your_reddit_username'
GEMINI_API_KEY = 'your_gemini_api_key'

# =================== SETUP ===================
import praw
import google.generativeai as genai

# 🔧 Configure Gemini API
genai.configure(api_key=GEMINI_API_KEY)

# 🔧 Set up Reddit client using PRAW
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_SECRET,
    user_agent=f"script:persona.generator:v1.0 (by u/{REDDIT_USERNAME})"
)

# =================== FETCH REDDIT DATA ===================
def fetch_user_data(username, post_limit=20, comment_limit=20):
    """
    Fetches recent posts and comments from a Reddit user.
    """
    try:
        redditor = reddit.redditor(username)
        posts, comments = [], []

        for submission in redditor.submissions.new(limit=post_limit):
            posts.append({
                "type": "post",
                "title": submission.title,
                "text": submission.selftext,
                "link": f"https://www.reddit.com{submission.permalink}"
            })

        for comment in redditor.comments.new(limit=comment_limit):
            comments.append({
                "type": "comment",
                "text": comment.body,
                "link": f"https://www.reddit.com{comment.permalink}"
            })

        return posts, comments

    except Exception as e:
        print(f"❌ Failed to fetch content for u/{username}: {e}")
        return [], []

# =================== GENERATE PERSONA ===================
def generate_persona(posts, comments):
    """
    Uses Gemini to analyze a user's posts/comments and generate a persona.
    """
    combined_text = ""
    for item in posts + comments:
        if item["type"] == "post":
            combined_text += f"[POST] {item['title']}\n{item['text']}\n(Source: {item['link']})\n\n"
        else:
            combined_text += f"[COMMENT] {item['text']}\n(Source: {item['link']})\n\n"

    prompt = f"""
Based on the following Reddit posts and comments, generate a detailed user persona including:
- Interests
- Personality Traits
- Political Views (if detectable)
- Writing Style and Tone

Cite specific posts/comments using (Source: ...) format.

--- BEGIN TEXT ---
{combined_text}
--- END TEXT ---
"""

    try:
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"❌ Gemini API Error: {e}")
        return ""

# =================== SAVE OUTPUT ===================
def save_to_file(username, content):
    """
    Saves the generated persona to a local text file.
    """
    file_name = f"{username}.txt"
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"✅ Persona saved to {file_name}")

# =================== MAIN FUNCTION ===================
def generate_reddit_persona(profile_url):
    """
    Orchestrates the entire flow: fetch user → analyze → save persona.
    """
    try:
        username = profile_url.strip("/").split("/")[-1]
        print(f"\n🔍 Generating persona for u/{username}...")

        posts, comments = fetch_user_data(username)

        if len(posts) + len(comments) < 5:
            print("⚠️ Not enough data to build a useful persona.")
            return

        print(f"📥 Fetched {len(posts)} posts and {len(comments)} comments.")

        persona = generate_persona(posts, comments)

        if not persona.strip():
            print("⚠️ Gemini returned empty or unusable output.")
            return

        save_to_file(username, persona)
        print("🎉 Persona generation complete!")

    except Exception as e:
        print(f"❌ Unexpected Error: {type(e).__name__} - {e}")

# =================== RUN GENERATOR ===================
# 🔁 Add more Reddit profiles to analyze here
generate_reddit_persona("https://www.reddit.com/user/kojied/")
generate_reddit_persona("https://www.reddit.com/user/Hungry-Move-6603/")


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




🔍 Generating persona for: kojied


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



❌ Failed to fetch content for kojied: received 401 HTTP response
⚠️ Not enough content to generate a useful persona.

🔍 Generating persona for: Hungry-Move-6603
❌ Failed to fetch content for Hungry-Move-6603: received 401 HTTP response
⚠️ Not enough content to generate a useful persona.
